In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
# Load the dataset
data = pd.read_csv("../data/tesla_data.csv", index_col='Date', parse_dates=True)
data = data[['Close']]  # Using only the 'Close' price for forecasting
data = data.resample('D').ffill()  # Resampling to fill any missing dates

In [ ]:
# Split data into train and test sets
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

# Scale the data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

In [ ]:
# Load the saved model
model_path = '../models/lstm_model.h5'
lstm_model = load_model(model_path, custom_objects={'mse': MeanSquaredError()})

In [ ]:
# Set forecasting parameters
n_input = 30
n_features = 1
future_periods = 180  # Predicting for 6 months (approx. 180 trading days)

# Prepare the first input batch for forecasting
forecast = []
first_eval_batch = train_scaled[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

# Forecast future prices
for _ in range(future_periods):
    lstm_pred = lstm_model.predict(current_batch)[0]
    forecast.append(lstm_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[lstm_pred]], axis=1)

# Inverse transform to obtain actual prices
forecast = scaler.inverse_transform(forecast)

# Prepare date range for the forecasted values
forecast_index = pd.date_range(start=test.index[-1] + pd.Timedelta(days=1), periods=future_periods, freq='D')

# Convert forecast to DataFrame for visualization
forecast_df = pd.DataFrame(data=forecast, index=forecast_index, columns=['Forecast'])

In [ ]:
# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(train.index, train['Close'], label='Training Data')
plt.plot(test.index, test['Close'], label='Testing Data')
plt.plot(forecast_df.index, forecast_df['Forecast'], label='Forecast')
plt.title('LSTM Model - Tesla Stock Price Forecast for 6 Months')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()


In [ ]:

#Forecast Analysis

# Trend Analysis
long_term_trend = forecast_df.rolling(window=30).mean()  # 30-day moving average as a proxy for trend
plt.figure(figsize=(12, 6))
plt.plot(forecast_df.index, forecast_df['Forecast'], label='Forecast')
plt.plot(long_term_trend, label='Long-term Trend', linestyle='--', color='orange')
plt.title('Trend Analysis for Tesla Stock Price Forecast')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

# Volatility and Confidence Intervals
# Calculate the standard deviation of forecasted values for confidence intervals
forecast_std = np.std(forecast_df['Forecast'])
upper_bound = forecast_df['Forecast'] + forecast_std
lower_bound = forecast_df['Forecast'] - forecast_std

plt.figure(figsize=(12, 6))
plt.plot(forecast_df.index, forecast_df['Forecast'], label='Forecast')
plt.fill_between(forecast_df.index, lower_bound, upper_bound, color='b', alpha=0.2, label='Confidence Interval')
plt.title('Tesla Stock Price Forecast with Confidence Interval')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

# Interpret Results
print("Trend Analysis:")
print("The forecasted trend suggests a potential long-term pattern where stock prices may be moving upward/downward.")
print("\nVolatility and Risk:")
print("The width of the confidence intervals indicates expected volatility in the forecasted period. Larger intervals suggest higher risk.")

# Market Opportunities and Risks
# Based on the forecast, summarize expected market opportunities or risks:
# Define a threshold for volatility (you can adjust this value based on your data and analysis)
volatility_threshold = 0.05  # or any other value that fits your risk assessment

# Check for market opportunity and risk
if upper_bound[-1] > forecast_df['Forecast'].iloc[-1]:
    print("Market Opportunity: There is potential for price increases, which may present buying opportunities.")
if forecast_std > volatility_threshold:  
    print("Market Risk: High volatility could mean increased investment risk, especially if unexpected events impact prices.")
